# 텐서와 그래프 실행

In [2]:
# 텐서플로 import
import tensorflow as tf

In [3]:
# 상수를 hello 변수에 저장
hello = tf.constant('Hello, TensorFlow!')
print(hello)

Tensor("Const:0", shape=(), dtype=string)


hello가 텐서플로의 **텐서** 라는 자료형을 담고 있다
**텐서는** 텐서플로에서 다양한 수학식을 계산하기 위한 가장 기본적이고 중요한 자료형이다.
다음과 같이 **랭크(Rank)** 와 **셰이프(Shape)** 라는 개념을 가지고 있다.

</bn>

```python
3                              # 랭크가 0인 텐서, 셰이프는 []
[1., 2., 3.]                   # 랭크가 1인 텐서, 셰이프는 [3]
[[1., 2., 3.], [4., 5., 6.]]   # 랭크가 2인 텐서, 셰이프는 [2, 3]
```
> 텐서 자료형의 형태는 배열과 비슷하다고 생각하면 된다.

</bn>

**dtype** 은 해당 텐서에 담긴 요소들의 자료형이다. ex) string, float, int


In [4]:
# 텐서를 이용한 덧셈
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a, b)
print(c)

Tensor("Add:0", shape=(), dtype=int32)


42가 나올 것으로 생각할 수 있지만, 위와 같이 텐서의 형태로 출력된다.
왜냐하면 텐서플로 프로그램의 구조는 **그래프의 생성과 그래프 실행** 으로 구성되어 있다.

<br>

* **그래프** : 텐서들의 연산 모음. 연산이 필요할 때만 실행하는 코드를 넣어 <u>원하는 시점</u> 에 실제 연산을 수행하도록 한다.
* **그래프 실행** : Session 객체와 run 메소드를 이용하면 된다.

In [5]:
sess = tf.Session()

print(sess.run(hello))
print(sess.run([a, b, c]))

sess.close

b'Hello, TensorFlow!'
[10, 32, 42]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x1315743c8>>

# 플레이스홀더와 변수
**플레이스홀더** : 그래프에 사용할 입력값을 나중에 받기 위해 사용하는 매개변수
**변수** : 그래프를 최적화하는 용도로 텐서플로가 학습한 결과를 갱신하기 위해 사용하는 변수

In [6]:
# 플레이스홀더 사용
X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


In [7]:
# X에 넣을 자료 정의
x_data = [[1, 2, 3], [4, 5, 6]]

In [8]:
# 변수 정의
# tf.random_normal : 정규분포의 무작위 값으로 최고화
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

In [9]:
# 행렬 곱셈 실행
expr = tf.matmul(X, W) + b

텐서플로로 딥러닝을 하기 위해서는 **행렬곱** 정의를 알아야 한다.
* 행렬곱 A x B 에 대하여, 행렬 A의 열 수와 행렬 B의 행 수는 같아야 한다.
* 행렬곱 A x B 를 계산한 행렬 AB의 크기는 A의 행 개수와 B의 열 개수가 된다.

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 정의한 변수들을 초기화

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
print(sess.run(expr, feed_dict={X: x_data})) # feed_dict : 그래프를 실행할 때 사용할 입력값을 지정

sess.close

=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[-0.63166183  0.9420165 ]
 [ 1.354897   -0.08405089]
 [-0.9487149  -1.0861019 ]]
=== b ===
[[-0.37491494]
 [-1.342244  ]]
=== expr ===
[[-1.1429274 -2.8593056]
 [-2.7866955 -4.5110435]]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x1315f2a20>>

# 선형 회귀 모델 구현하기
**선형 회귀** : 주어진 x 와 y 값을 가지고 서로 간의 관계를 파악하는 것입니다. x 값이 주어졌을 때 y 값을 쉽게 알 수 있다.

<br>
    
x_data 와 y_data의 상관관계를 파악해보자.

In [11]:
x_data = [1, 2, 3]
y_data = [1, 2, 3]

In [12]:
# -1.0 ~ 1.0 사이의 균등분포를 가진 무작위 값으로 초기화
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

In [13]:
# 자료를 입력받을 플레이스홀더를 설정
X = tf.placeholder(tf.float32, name="X") # name 매개변수로 플레이스홀더의 이름을 설정한다.
Y = tf.placeholder(tf.float32, name="Y")

print(X)
print(Y)

Tensor("X:0", dtype=float32)
Tensor("Y:0", dtype=float32)


In [14]:
# X와 Y의 상관관계(선형관계)를 분석하기 위한 수식 작성
hypothesis = W * X + b

이 수식은 W와의 곱과 b와의 합을 통해 **X와 Y의 관계를** 설명하겠다는 뜻이다.
즉, X가 주어졌을 때 Y를 만들어 낼 수 있는 W와 b를 찾아내겠다는 의미이다.
* **W** : 가중치
* **b** : 편향
> 이 수식은 선형 회귀는 물론 신경망 학습에 가장 기본이 되는 수식이다.

<br>

손실 함수를 작성해보자.
* **손실 함수(loss function)** : 한 쌍(x,y)의 데이터에 대한 **손실값**을 계산하는 함수이다.
* 손실값 : 실제값과 모델로 예측한 값이 얼마나 차이가 나는지를 나타내는 값
> 손실값이 작을수록 X와 Y의 관계가 잘 설명되고 있는 것이며, X 값에 대한 Y 값을 정확하게 예측할 수 있다는 뜻이다.
* **비용(cost)** : 손실을 전체 데이터에 대해 구한 경우
> 즉 **학습**이란 변수들의 값을 다양하게 넣어 계산해보면서 이 손실값을 최소화하는 것이다.

In [15]:
# 손실값 = (예측값 - 실제값)^2
# 모든 데이터에 대한 손실값의 평균을 낸다.
cost = tf.reduce_mean(tf.square(hypothesis - Y))

텐서플로 함수을 볼 수 있는 사이트
[https://www.tensorflow.org/api_docs/](https://www.tensorflow.org/api_docs/)

<br>

## 경사하강법 (gradient descent)
최적화 함수를 이용해 손실값을 최소화하는 연산 그래프 생성
* **최적화 함수** : 가중치와 편향 값을 변경해가면서 손실값을 최소화하는 가장 최적화된 가중치와 편향 값을 찾아주는 함수.
* **경사하강법** : 빠르게 최적화하기 위한, 즉 빠르게 학습하기 위한 다양한 방법 중에 가장 기본적인 알고리즘이다.
* **learning_rate(학습률)** : 학습을 얼마나 '급하게' 할 것인가를 설정하는 값
  * 값이 너무 작으면 학습 속도가 매우 느려지고, 너무 크면 최적의 손실값을 찾지 못한다.
* **하이퍼파라미터(hyperparameter)** : 학습을 진행하는 과정에 영향을 주는 변수. 이 값에 따라 학습 속도나 신경망 성능이 크게 달라진다.

In [18]:
# 최적화 함수를 이용해 손실값을 최소화하는 연산 그래프 생성 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(cost)

In [23]:
# 세션을 생성하고 변수들 초기화
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 최적화 수행 그래프 실행, 실행 시마다 변화하는 손실값 출력
    for step in range(100):
        # feed_dict 를 통해 x_data 와 y_data 의 상관관계를 알아낸다
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})
        
        print(step, cost_val, sess.run(W), sess.run(b))
        
    # X 값을 넣고 결과 확인
    print("\nX: 5, Y: ", sess.run(hypothesis, feed_dict={X: 5}))
    print("X: 2.5, Y: ", sess.run(hypothesis, feed_dict={X: 2.5}))
        
# 결과를 보면 손실값이 점점 줄어드는것을 확인할 수 있다.
# 그리고, X가 5일 때는 Y 값으로 5를 2.5일 때는 2.5를 정확히 예측해내는것을 ㅎ

0 26.296478 [1.2209363] [0.08877015]
1 0.31412363 [0.979221] [-0.0173584]
2 0.0037589802 [1.005558] [-0.00557513]
3 5.1295337e-05 [1.0026006] [-0.0066833]
4 6.7053843e-06 [1.0028467] [-0.00638685]
5 5.8834253e-06 [1.0027446] [-0.00624817]
6 5.597636e-06 [1.0026823] [-0.00609635]
7 5.3321396e-06 [1.0026174] [-0.00595001]
8 5.078769e-06 [1.0025545] [-0.00580696]
9 4.8377087e-06 [1.002493] [-0.00566739]
10 4.6077694e-06 [1.0024332] [-0.00553112]
11 4.3887358e-06 [1.0023746] [-0.00539817]
12 4.1801827e-06 [1.0023175] [-0.00526839]
13 3.981929e-06 [1.0022619] [-0.00514175]
14 3.7926256e-06 [1.0022075] [-0.00501815]
15 3.6124454e-06 [1.0021545] [-0.00489752]
16 3.4409766e-06 [1.0021026] [-0.00477981]
17 3.2774744e-06 [1.0020521] [-0.00466489]
18 3.1217762e-06 [1.0020027] [-0.00455274]
19 2.9733249e-06 [1.0019547] [-0.00444328]
20 2.8324848e-06 [1.0019076] [-0.0043365]
21 2.6975715e-06 [1.0018618] [-0.00423224]
22 2.5696472e-06 [1.001817] [-0.00413051]
23 2.4474696e-06 [1.0017734] [-0.0040312